In [1]:
import os
import glob
import soundfile as sf
import librosa
import random 

DATA_PATH = '/data3/VCTK/VCTK-Corpus/wav48'
TXT_PATH = '/data3/VCTK/VCTK-Corpus/txt'

ALL_PATH = sorted(glob.glob(os.path.join(DATA_PATH, '*/*.wav')))
ALL_TXT_PATH = sorted(glob.glob(os.path.join(TXT_PATH, '*/*.txt')))
SR = 16000
CUT = 12

In [21]:
# data, sr = sf.read('/data3/VCTK/VCTK-Corpus/wav48/p225/p225_001_backup.wav')
# print('sr', sr)

## Rewrite all the audio to 16k

In [20]:
# for path in ALL_PATH:
#     y, sr = librosa.load(path, sr=48000)
#     y_16k = librosa.resample(y, sr, 16000)
#     librosa.output.write_wav(path, y_16k, 16000)

In [2]:
for path in ALL_PATH:
    data, sr = sf.read(path)
    if sr != 16000:
        print('Something is wrong')
print('Scanned all the files')

Scanned all the files


## Add files to the train list

In [17]:
def read_text(path):
    with open(path, 'r') as f:
        line = f.readline()
        if '\n' in line:
            return line
        else:
            line = line + '\n'
            return line

    
def check_audio_length(path):
    audio, _ = sf.read(path)
    if len(audio) < (CUT * SR):
        return True
    else:
        return False

    
def random_sample(used_path, all_path):
    all_path.remove(used_path)
    path = random.sample(all_path, 1)[0]
    while True:
        if check_audio_length(path) == False:
            path = random.sample(all_path, 1)[0]
        else:
            return path

In [18]:
train_path = '/home/xliu0/workspace/tacotron2/filelists/train_filelist_vctk.txt'
train_alt_path = '/home/xliu0/workspace/tacotron2/filelists/train_filelist_alt.txt'
# train_path = '/home/xliu0/workspace/tacotron2/filelists/IEMOCAP/test.txt'
# train_alt_path = '/home/xliu0/workspace/tacotron2/filelists/IEMOCAP/test_alt.txt'

for path, txt_path in zip(ALL_PATH, ALL_TXT_PATH):
    if path.split('/')[-1].split('.')[0] == txt_path.split('/')[-1].split('.')[0]:  
        # read data
        data, sr = sf.read(path)
        if len(data) > (SR * CUT):
            continue
        else:
            
            with open(train_path, 'a') as f1, open(train_alt_path, 'a') as f2:
                # Write into train file:
                s1 = path + '|' + read_text(txt_path)
                f1.write(s1)
                # Write into alt file 
                data_folder_path = path.split('/')[:-1]
                data_folder_path = '/'.join(data_folder_path)
                all_wav_data_folder = sorted(glob.glob(data_folder_path + '/*.wav'))
                alt_wav_path = random_sample(path, all_wav_data_folder)
                alt_txt_path = alt_wav_path.split('/')
                alt_txt_path[4] = 'txt'
                alt_txt_path = '/'.join(alt_txt_path).split('.')[0] + '.txt'
                s2 = alt_wav_path + '|' + read_text(alt_txt_path)
                f2.write(s2)                      
